```python
import tdmclient.notebook
await tdmclient.notebook.list()
await tdmclient.notebook.start()


```python
%%run_python
nf_leds_temperature(0,0)
nf_leds_prox_v(0,0)

In [3]:
from time import sleep
from tdmclient import ClientAsync, aw

def get_status(node):
    status = node.status
    if status == 1:
        return "Connected"
    if status == 2:
        return "Unlocked"
    if status == 3:
        return "Busy"
    if status == 4:
        return "Locked"
    if status == 5:
        return "Disconnected"
    else:
        return "Unknown"

def lock_node(node):
    aw(node.lock())
    sleep(0.1)
    get_status(node)
               
def unlock_node(node):
    aw(node.unlock())
    sleep(0.1)
    get_status(node)

try:
    client = ClientAsync()
    node = aw(client.wait_for_node())
    sleep(0.1)
    
    if node.status != 2:
        print("Please unlock the thymio in Aseba Studio or check connection")
    
    aw(node.wait_for_variables())
    
    try:
        aw(node.lock())
        sleep(0.1)
        aw(node.unlock())
    except:
        print("Please unlock Thymio in Aseba Studio")

    sleep(0.1)
    print(node.props["name"], get_status(node))
except:
    print("Open Aseba Studio and Unlock Thymio / reconnect Thymio")

Super Thymio 474 Unlocked


In [11]:
#vars(node)

In [9]:
import numpy as np

PROX_FARLEFT = 0
PROX_LEFT = 1
PROX_FRONT = 2
PROX_RIGHT = 3
PROX_FARRIGHT = 4
PROX_BACKLEFT = 5
PROX_BACKRIGHT = 6

def get_prox(node):
    prox_farleft = node.v.prox.horizontal[PROX_FARLEFT]
    prox_left = node.v.prox.horizontal[PROX_LEFT]
    prox_front = node.v.prox.horizontal[PROX_FRONT]
    prox_right = node.v.prox.horizontal[PROX_RIGHT]
    prox_farright = node.v.prox.horizontal[PROX_FARRIGHT]
    prox_backleft = node.v.prox.horizontal[PROX_BACKLEFT]
    prox_backright = node.v.prox.horizontal[PROX_BACKRIGHT]
    
    # just showing if you need numpy, you could do a straightforward mean without numpy
    prox_array = np.array([prox_farleft, prox_left, prox_front, prox_right, prox_farright, prox_backleft, prox_backright]) # (7,)
    #print(np.mean(prox_array))
    if np.mean(prox_array) != 0:
        return True
    else:
        return False

def motors(node, speed_left, speed_right):
    node.v.motor.left.target = left
    node.v.motor.right.target = right

In [10]:
lock_node(node)

speed = 100
manual_drive = False

while True:
    if manual_drive:
        node.v.leds.top = [32,32,0]
        node.v.leds.circle = [0,0,0,0,0,0,0,0]
        
        if node.v.button.backward == 1:
            motors(node,-speed,-speed)
            node.v.leds.circle = [0,0,0,0,32,0,0,0]
    
        if node.v.button.left == 1:
            motors(node,-speed,speed)
            node.v.leds.circle = [0,0,0,0,0,0,32,0]
    
    
        if node.v.button.right == 1:
            motors(node,speed,-speed)
            node.v.leds.circle = [0,0,32,0,0,0,0,0]
            
        if node.v.button.forward == 1:
            motors(node,speed,speed)
            node.v.leds.circle = [32,0,0,0,0,0,0,0]
    else:
        node.v.leds.circle = [0,0,0,0,0,0,0,0]
        if get_prox(node):
            motors(node,0,0)
            node.v.leds.top = [32,0,0]
            node.flush()
        else:
            motors(node,speed,speed)
            node.v.leds.top = [0,32,0]

    if node.v.button.center == 1:
        print("Exiting...")
        break
    node.flush()
    #await client.sleep(0.1)

unlock_node(node)

Exiting...
